In [52]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from multiprocessing import cpu_count

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Subset, DataLoader

from torch.distributions import *

import skorch
import numpy as np

%load_ext tensorboard
torch.autograd.set_detect_anomaly(True)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Dataset

In [31]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': cpu_count(), 'pin_memory': True} if use_cuda else {}
train_loader = DataLoader(Subset(
    datasets.MNIST('/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), indices=range(10000)),
    batch_size=64, shuffle=True, **kwargs)
test_loader = DataLoader(Subset(
    datasets.MNIST('/data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), indices=range(10000)),
    batch_size=1000, shuffle=True, **kwargs)

## Generic autoencoder class

In [91]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.writer = SummaryWriter(log_dir='/data/runs')
    
    def trains(self, device, train_loader, optimizer, epoch):
        self.train()
        loss_sum = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = self.compute_loss_train(data, target)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
            self.writer.add_scalar('Loss/train', loss.item(), epoch*len(train_loader)+batch_idx)
            
    def tests(self, device, test_loader):
        self.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                loss, output = self.compute_loss_test(data, target)
                test_loss += loss
                l1 = F.l1_loss(output, data.view(-1, 784), reduction='sum')

        test_loss /= len(test_loader.dataset)

        print('\nTest set: Average loss: {:.4f}, Reconstruction error: {}\n'.format(
            test_loss, l1))

## Autoencoder

In [92]:
class SimpleAutoencoder(Autoencoder):
    def __init__(self):
        super(SimpleAutoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output       
    
    def compute_loss_train(self, data, target):
        output = self(data)
        return F.nll_loss(output, target)
    
    def compute_loss_test(self, data, target):
        output = self(data)
        return F.nll_loss(output, target, reduction='sum').item(), output  # sum up batch loss

In [80]:
model = SimpleAutoencoder().to(device)
optimizer = optim.Adadelta(model.parameters())

# plot model
dataiter = iter(train_loader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# write to tensorboard
#writer.add_image('mnist_images', img_grid)

scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 2.325908


KeyboardInterrupt: 

## Gaussian Variational Autoencoder

In [93]:
class VAE(Autoencoder):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(9216, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    def loss_function(self, recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD
    
    def compute_loss_train(self, data, target):
        recon_batch, mu, logvar = self(data)
        return self.loss_function(recon_batch, data, mu, logvar)
    
    def compute_loss_test(self, data, target):
        recon_batch, mu, logvar = self(data)
        return self.loss_function(recon_batch, data, mu, logvar).item(), recon_batch  # sum up batch loss

In [94]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 35111.679688
Train Epoch: 1 [640/10000 (6%)]	Loss: -100550.734375
Train Epoch: 1 [1280/10000 (13%)]	Loss: -290526.843750
Train Epoch: 1 [1920/10000 (19%)]	Loss: -367270.281250
Train Epoch: 1 [2560/10000 (25%)]	Loss: -398255.500000
Train Epoch: 1 [3200/10000 (32%)]	Loss: -438068.968750
Train Epoch: 1 [3840/10000 (38%)]	Loss: -512162.750000
Train Epoch: 1 [4480/10000 (45%)]	Loss: -534456.562500
Train Epoch: 1 [5120/10000 (51%)]	Loss: -564552.687500
Train Epoch: 1 [5760/10000 (57%)]	Loss: -667109.625000
Train Epoch: 1 [6400/10000 (64%)]	Loss: -716329.500000
Train Epoch: 1 [7040/10000 (70%)]	Loss: -738120.000000
Train Epoch: 1 [7680/10000 (76%)]	Loss: -718507.750000
Train Epoch: 1 [8320/10000 (83%)]	Loss: -751547.312500
Train Epoch: 1 [8960/10000 (89%)]	Loss: -861383.875000
Train Epoch: 1 [9600/10000 (96%)]	Loss: -846808.000000

Test set: Average loss: -13556.3616, Reconstruction error: 495748.84375

Train Epoch: 2 [0/10000 (0%)]	Loss: -854750.750000
Tra

KeyboardInterrupt: 

## Stick-breaking process

In [95]:
def stickbreakingprocess(k, a, b):
    batch_size = a.size()[0]
    uniform_samples = Uniform(torch.tensor([0.0]), torch.tensor([1.0])).rsample(torch.tensor([batch_size,k])).view(-1,k)
    exp_a = 1/a
    exp_b = 1/b
    km = (1- uniform_samples.pow(exp_b)).pow(exp_a)
    
    #no Nans are allowed in the matrix
    #assert not torch.isnan(km).any().item()
    
    sticks = torch.zeros(batch_size,k)
    remaining_sticks = torch.ones_like(km[:,0])
    with torch.no_grad():
        for i in range(0,k-1):
            sticks[:,i] = remaining_sticks * km[:,i]
            remaining_sticks *= (1-km[:,i])
        sticks[:,k-1] = remaining_sticks
    latent_variables = sticks

    #all stick segments must sum to 1
    #assert torch.allclose(latent_variables.sum(axis=1), torch.ones([batch_size]))
    
    return latent_variables

In [96]:
stickbreakingprocess(20, torch.rand(10,20), torch.rand(10,20)).sum(axis=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [97]:
t = torch.rand([5,10])
t.unfold(1,2,2)

tensor([[[0.2840, 0.0518],
         [0.7416, 0.3510],
         [0.0801, 0.8012],
         [0.3975, 0.7871],
         [0.9122, 0.6335]],

        [[0.7045, 0.6593],
         [0.6110, 0.3700],
         [0.1818, 0.2044],
         [0.3644, 0.4365],
         [0.9949, 0.6306]],

        [[0.4014, 0.6926],
         [0.9259, 0.5556],
         [0.2471, 0.2153],
         [0.0361, 0.9757],
         [0.1972, 0.3511]],

        [[0.7728, 0.4319],
         [0.5881, 0.0910],
         [0.4601, 0.1714],
         [0.9667, 0.2084],
         [0.9598, 0.7851]],

        [[0.2760, 0.4266],
         [0.4791, 0.1536],
         [0.9422, 0.8401],
         [0.3884, 0.9252],
         [0.1256, 0.6673]]])

## Stick-breaking Autoencoder

In [98]:
class SBVAE(Autoencoder):
    def __init__(self, k):
        super(SBVAE, self).__init__()
        self.k = k
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(9216, 400)
        self.fc21 = nn.Linear(400, k)
        self.fc22 = nn.Linear(400, k)
        
        
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        h1 = F.relu(self.fc1(x))
        return F.softplus(self.fc21(h1)), F.softplus(self.fc22(h1))

    def reparameterize(self, a, b):
        return stickbreakingprocess(20, a, b)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        a, b = self.encode(x)
        z = self.reparameterize(a, b)
        return self.decode(z), a, b
    
    def Beta(self, a,b):
        return torch.exp(torch.lgamma(a) + torch.lgamma(b) - torch.lgamma(a+b))
    
    def loss_function(self, recon_x, x, a, b, prior_alpha, prior_beta):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
        kl = 1./(1+a*b) * self.Beta(1./a, b)
        kl += 1./(2+a*b) * self.Beta(2./a, b)
        kl += 1./(3+a*b) * self.Beta(3./a, b)
        kl += 1./(4+a*b) * self.Beta(4./a, b)
        kl += 1./(5+a*b) * self.Beta(5./a, b)
        kl += 1./(6+a*b) * self.Beta(6./a, b)
        kl += 1./(7+a*b) * self.Beta(7./a, b)
        kl += 1./(8+a*b) * self.Beta(8./a, b)
        kl += 1./(9+a*b) * self.Beta(9./a, b)
        kl += 1./(10+a*b) * self.Beta(10./a, b)
        kl *= (prior_beta-1)*b
                                                                                                                                            
        kl += (a-prior_alpha)/a * (-np.euler_gamma - torch.digamma(b) - 1/b) #T.psi(self.posterior_b)                                                                                        

        # add normalization constants                                                                                                                                                                
        kl += torch.log(a*b) #+ torch.log(self.Beta(prior_alpha, prior_beta))

        # final term                                                                                                                                                                                 
        kl += -(b-1)/b 

        return BCE + kl.sum()
    
    def compute_loss_train(self, data, target):
        recon_batch, a, b = self(data)
        return self.loss_function(recon_batch, data, a, b, torch.Tensor([1]), torch.Tensor([5]))
    
    def compute_loss_test(self, data, target):
        recon_batch, a, b = self(data)
        return self.loss_function(recon_batch, data, a, b, 1, 5).item(), recon_batch  # sum up batch loss

In [99]:
model = SBVAE(k=20).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1): 
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 40108.292969
Train Epoch: 1 [640/10000 (6%)]	Loss: 28535.388672
Train Epoch: 1 [1280/10000 (13%)]	Loss: 18639.339844
Train Epoch: 1 [1920/10000 (19%)]	Loss: 6732.952148
Train Epoch: 1 [2560/10000 (25%)]	Loss: -5551.542969
Train Epoch: 1 [3200/10000 (32%)]	Loss: -21250.224609
Train Epoch: 1 [3840/10000 (38%)]	Loss: -28925.025391
Train Epoch: 1 [4480/10000 (45%)]	Loss: -45837.699219
Train Epoch: 1 [5120/10000 (51%)]	Loss: -65902.265625
Train Epoch: 1 [5760/10000 (57%)]	Loss: -88064.898438
Train Epoch: 1 [6400/10000 (64%)]	Loss: -102332.882812
Train Epoch: 1 [7040/10000 (70%)]	Loss: -141090.687500
Train Epoch: 1 [7680/10000 (76%)]	Loss: -173426.468750
Train Epoch: 1 [8320/10000 (83%)]	Loss: -206034.984375
Train Epoch: 1 [8960/10000 (89%)]	Loss: -255373.250000
Train Epoch: 1 [9600/10000 (96%)]	Loss: -306094.718750

Test set: Average loss: -5136.3508, Reconstruction error: 564488.9375

Train Epoch: 2 [0/10000 (0%)]	Loss: -328507.937500
Train Epoch: 2 [640